In [1]:
import pandas as pd
df = pd.read_csv('./data/rg65.txt', delimiter =  ";", header = None)
df.columns = ['word1', 'word2', 'org_sim']

In [2]:
df['org_sim'] = df['org_sim'] / 4

In [3]:
# mm_model_path = 'multimodal_lstm_200000_1.0_15v6'#'multimodal_lstm_200000_1.0_15v5'##'multimodal_lstm_200000_0.7071067811865475_15v2'
# bm_model_path = 'benchmark_model_200000_1.0_15v7'#benchmark_model_200000_1.0_15v6'#'benchmark_model_200000_0.7071067811865475_15v4'

In [4]:
mm_model_path = "finetuned_multimodal_lstm"#'multimodal_lstm_200000_1.0_15v6'#'multimodal_lstm_200000_1.0_15v5'##'multimodal_lstm_200000_0.7071067811865475_15v2'
bm_model_path = 'finetuned_benchmark_lstm'#benchmark_model_200000_1.0_15v6'#'benchmark_model_200000_0.7071067811865475_15v4'

In [5]:
import torch
device = 'cuda' if torch.cuda.is_available() else "gpu"
from multimodal_model import BenchmarkLSTM, MMLSTM
from multi_bpe import MultiBPE
multi_bpe = MultiBPE()
def load_model(loaded_model, is_benchmark, is_multimodal, train_visual_module):
    if is_benchmark:
        mm_model = BenchmarkLSTM().to(device)
    else:
        mm_model = MMLSTM(is_multimodal=is_multimodal,
                     train_visual_module=train_visual_module).to(device)
    
    mm_model.load_state_dict(torch.load(f'./saved_models/{loaded_model}'))
    mm_model.eval()
    return mm_model

mm_model = load_model(mm_model_path, False, True, False)
bm_model = load_model(bm_model_path, True, False, False)

In [6]:
def cosine_score(w1, w2, model, name='emb_layer.weight'):
    tcs = torch.nn.CosineSimilarity(dim=1)
    e1 = multi_bpe.encode(w1, False, False, None)
    e2 = multi_bpe.encode(w2, False, False, None)
    if len(e1) > 1 or len(e2) > 1:
        a = model.state_dict()[name][e1[0]]
        for idx in e1[1:]:
            a = torch.concat((a, model.state_dict()[name][idx]))
        b = model.state_dict()[name][e2[0]]
        for idx in e2[1:]:
            b = torch.concat((b, model.state_dict()[name][idx]))
        return 999
        return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()
    else:
        a = model.state_dict()[name][e1[0]]
        b = model.state_dict()[name][e2[0]]
        return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()

def cosine_sim_model(df, model, name):
    l = []    
    for i in range(len(df)):
        w1, w2 = df.word1[i], df.word2[i]
        l.append(cosine_score(w1, w2, model, name))
    return l

In [7]:
df['bm_sim'] = cosine_sim_model(df, bm_model, 'fc.weight')
df['mm_sim'] = cosine_sim_model(df, mm_model, 'fc.weight')

<ipython-input-6-b642a3c19c31>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()
<ipython-input-6-b642a3c19c31>:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return tcs(torch.tensor(a).view(1,-1),torch.tensor(b).view(1,-1)).item()


In [8]:
df = df[df.bm_sim != 999].reset_index(drop=True)
df

,word1,word2,org_sim,bm_sim,mm_sim
0,cord,smile,0.0050,0.867759,0.890492
1,noon,string,0.0100,0.866918,0.840501
2,automobile,wizard,0.0275,0.994545,0.994508
3,grin,implement,0.0450,0.901850,0.872949
4,asylum,fruit,0.0475,0.834259,0.830115
5,asylum,monk,0.0975,0.997391,0.995776
6,monk,slave,0.1425,0.997177,0.995932
7,asylum,cemetery,0.1975,0.987907,0.992160
8,coast,forest,0.2125,0.883385,0.877831
9,grin,lad,0.2200,0.996050,0.996309


In [9]:
df.corr(method='pearson')

,org_sim,bm_sim,mm_sim
org_sim,1.000000,-0.303307,-0.330001
bm_sim,-0.303307,1.000000,0.965955
mm_sim,-0.330001,0.965955,1.000000


In [10]:
import pingouin as pg

#find partial correlation between hours and exam score while controlling for grade
pg.partial_corr(data=df, x='mm_sim', y='org_sim', covar='bm_sim')

,n,r,CI95%,p-val
pearson,41,-0.15017,"[-0.44, 0.17]",0.355018


In [11]:
0.335244**2

0.11238853953599999

In [12]:
0.426371**2

0.181792229641

In [13]:
0.326276**2

0.106456028176